In [1]:
import os
import json
import redis
from redisgraph import Graph, Node, Edge
import matplotlib.pyplot as plt
import networkx as nx
from matplotlib.font_manager import FontProperties

font_path = "/Users/naokiyoshimaru/Downloads/ipaexg00401/ipaexg.ttf"
font_prop = FontProperties(fname=font_path)

In [3]:
redis_con = redis.Redis(host='localhost', port=6379)
graph = Graph('KyotoCityGraph', redis_con)

In [6]:
baseSpot_result.result_set

[]

In [4]:
base_station = "今出川駅"
base_query = f"MATCH (n:Station {{name: '{base_station}'}}) RETURN n"

baseSpot_result = graph.query(base_query)
baseSpot_node_properties = baseSpot_result.result_set[0][0].properties
baseSpot_name = baseSpot_node_properties['name']
baseSpot_latitude = baseSpot_node_properties['latitude']
baseSpot_longitude = baseSpot_node_properties['longitude']

G = nx.Graph()

G.add_node(baseSpot_name, latitude=baseSpot_latitude, longitude=baseSpot_longitude)


IndexError: list index out of range

In [5]:


query = f"MATCH (n:Station {{name: '{base_station}'}})--(connected_node) RETURN connected_node"
result = graph.query(query)
# クエリの結果からノードとエッジを追加
for record in result.result_set:
    connected_node_properties = record[0].properties
    name = connected_node_properties['name']
    latitude = connected_node_properties['latitude']
    longitude = connected_node_properties['longitude']
    G.add_node(name, latitude=latitude, longitude=longitude)
    G.add_edge(base_station, name)


# 位置情報から描画位置を計算
pos = {node: (float(data['longitude']), float(data['latitude'])) for node, data in G.nodes(data=True)}

plt.figure(figsize=(10,10))
# グラフを描画
nx.draw(G, pos, with_labels=False, node_size=500, node_color="skyblue")

# ラベルの描画
for node, (x, y) in pos.items():
    plt.text(x, y, node, fontproperties=font_prop, ha='center')
    
plt.title(f'{base_station}周辺の観光地', fontproperties=font_prop)
plt.xlabel('経度', fontproperties=font_prop)
plt.ylabel('緯度', fontproperties=font_prop)
plt.show()


NameError: name 'G' is not defined